<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Lab: Custom Training Loops in Keras**


Estimated time needed: **30** minutes


In this lab, you will learn to implement a basic custom training loop in Keras. 


## Objectives

By the end of this lab, you will: 

- Set up the environment 

- Define the neural network model 

- Define the Loss Function and Optimizer 

- Implement the custom training loop 

- Enhance the custom training loop by adding an accuracy metric to monitor model performance 

- Implement a custom callback to log additional metrics and information during training


----


## Step-by-Step Instructions:


### Exercise 1: Basic custom training loop: 

#### 1. Set Up the Environment:

- Import necessary libraries. 

- Load and preprocess the MNIST dataset. 


In [1]:
# !pip install tensorflow numpy

In [2]:
import os
import warnings
import tensorflow as tf 
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input
from keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


I0000 00:00:1739876607.367631   16590 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739876607.717642   16590 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739876607.718122   16590 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739876607.726309   16590 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739876607.726572   16590 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

#### 2. Define the model: 

Create a simple neural network model with a Flatten layer followed by two Dense layers. 


In [3]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])


#### 3. Define Loss Function and Optimizer: 

- Use Sparse Categorical Crossentropy for the loss function. 
- Use the Adam optimizer. 


In [4]:
# Step 3: Define Loss Function and Optimizer
import keras

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = keras.optimizers.Adam()


#### 4. Implement the Custom Training Loop: 

- Iterate over the dataset for a specified number of epochs. 
- Compute the loss and apply gradients to update the model's weights. 


In [5]:
# Step 4: Implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.457242012023926
Epoch 1 Step 200: Loss = 0.3836127817630768
Epoch 1 Step 400: Loss = 0.1890982836484909
Epoch 1 Step 600: Loss = 0.1462833285331726
Epoch 1 Step 800: Loss = 0.14003729820251465
Epoch 1 Step 1000: Loss = 0.4424075484275818
Epoch 1 Step 1200: Loss = 0.19101178646087646
Epoch 1 Step 1400: Loss = 0.23593319952487946
Epoch 1 Step 1600: Loss = 0.21617260575294495
Epoch 1 Step 1800: Loss = 0.18259470164775848


2025-02-18 14:36:02.998489: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Start of epoch 2
Epoch 2 Step 0: Loss = 0.11401508748531342
Epoch 2 Step 200: Loss = 0.1742154210805893
Epoch 2 Step 400: Loss = 0.09956812858581543
Epoch 2 Step 600: Loss = 0.06266943365335464
Epoch 2 Step 800: Loss = 0.08235393464565277
Epoch 2 Step 1000: Loss = 0.30628886818885803
Epoch 2 Step 1200: Loss = 0.10803788900375366
Epoch 2 Step 1400: Loss = 0.15875114500522614
Epoch 2 Step 1600: Loss = 0.1807175874710083
Epoch 2 Step 1800: Loss = 0.08138629794120789


2025-02-18 14:49:53.326274: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Exercise 2: Adding Accuracy Metric:

Enhance the custom training loop by adding an accuracy metric to monitor model performance. 

#### 1. Set Up the Environment: 

Follow the setup from Exercise 1. 


In [6]:
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model: 
Use the same model as in Exercise 1. 


In [7]:
# Step 2: Define the Model

model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define the loss function, optimizer, and metric: 

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer. 

- Add Sparse Categorical Accuracy as a metric. 


In [8]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with accuracy: 

Track the accuracy during training and print it at regular intervals. 


In [9]:
# Step 4: Implement the Custom Training Loop with Accuracy

epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.4062142372131348 Accuracy = 0.09375
Epoch 1 Step 200: Loss = 0.4333232045173645 Accuracy = 0.8322450518608093
Epoch 1 Step 400: Loss = 0.21356435120105743 Accuracy = 0.8652587532997131
Epoch 1 Step 600: Loss = 0.20134828984737396 Accuracy = 0.8817075490951538
Epoch 1 Step 800: Loss = 0.17993290722370148 Accuracy = 0.8941947817802429
Epoch 1 Step 1000: Loss = 0.3941946029663086 Accuracy = 0.9017232656478882
Epoch 1 Step 1200: Loss = 0.17823797464370728 Accuracy = 0.9082015156745911
Epoch 1 Step 1400: Loss = 0.18994411826133728 Accuracy = 0.91285240650177
Epoch 1 Step 1600: Loss = 0.19023583829402924 Accuracy = 0.9160876274108887
Epoch 1 Step 1800: Loss = 0.1969292014837265 Accuracy = 0.9200791120529175
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07561000436544418 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.14596650004386902 Accuracy = 0.9605099558830261
Epoch 2 Step 400: Loss = 0.11515749990940094 Accuracy = 0.9568266868591309
Epoch 2 Step 600: Loss

2025-02-18 14:53:04.252731: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Start of epoch 3
Epoch 3 Step 0: Loss = 0.03599400818347931 Accuracy = 1.0
Epoch 3 Step 200: Loss = 0.053903985768556595 Accuracy = 0.9745025038719177
Epoch 3 Step 400: Loss = 0.11207377165555954 Accuracy = 0.9710879325866699
Epoch 3 Step 600: Loss = 0.03598571568727493 Accuracy = 0.9721817970275879
Epoch 3 Step 800: Loss = 0.06041228026151657 Accuracy = 0.9730805158615112
Epoch 3 Step 1000: Loss = 0.14074906706809998 Accuracy = 0.9739323258399963
Epoch 3 Step 1200: Loss = 0.03493092954158783 Accuracy = 0.9742141962051392
Epoch 3 Step 1400: Loss = 0.03935158625245094 Accuracy = 0.9747501611709595
Epoch 3 Step 1600: Loss = 0.08066152036190033 Accuracy = 0.9746252298355103
Epoch 3 Step 1800: Loss = 0.09462615102529526 Accuracy = 0.9751526713371277
Start of epoch 4
Epoch 4 Step 0: Loss = 0.020145533606410027 Accuracy = 1.0
Epoch 4 Step 200: Loss = 0.056998834013938904 Accuracy = 0.9825870394706726
Epoch 4 Step 400: Loss = 0.0750744491815567 Accuracy = 0.9807512760162354
Epoch 4 Step 600: 

### Exercise 3: Custom Callback for Advanced Logging: 

Implement a custom callback to log additional metrics and information during training. 

#### 1. Set Up the Environment: 

Follow the setup from Exercise 1.


In [10]:
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model: 

Use the same model as in Exercise 1. 


In [11]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define Loss Function, Optimizer, and Metric: 

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer. 

- Add Sparse Categorical Accuracy as a metric. 


In [12]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with custom callback: 

Create a custom callback to log additional metrics at the end of each epoch.


In [14]:
from keras.callbacks import Callback

# Step 4: Implement the Custom Callback 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')


In [15]:
# Step 5: Implement the Custom Training Loop with Custom Callback

epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.4232845306396484 Accuracy = 0.09375
Epoch 1 Step 200: Loss = 0.40861478447914124 Accuracy = 0.8327114582061768
Epoch 1 Step 400: Loss = 0.17375752329826355 Accuracy = 0.865882158279419
Epoch 1 Step 600: Loss = 0.12971322238445282 Accuracy = 0.8825395107269287
Epoch 1 Step 800: Loss = 0.15798711776733398 Accuracy = 0.8950920701026917
Epoch 1 Step 1000: Loss = 0.4190743565559387 Accuracy = 0.9025974273681641
Epoch 1 Step 1200: Loss = 0.1682654321193695 Accuracy = 0.9093984365463257
Epoch 1 Step 1400: Loss = 0.2221561074256897 Accuracy = 0.9146145582199097
Epoch 1 Step 1600: Loss = 0.19532829523086548 Accuracy = 0.9176491498947144
Epoch 1 Step 1800: Loss = 0.21630579233169556 Accuracy = 0.9215192794799805


2025-02-18 15:12:28.405362: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


End of epoch 1, loss: 0.039623018354177475, accuracy: 0.9235166907310486
Start of epoch 2
Epoch 2 Step 0: Loss = 0.06760856509208679 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.140646830201149 Accuracy = 0.9609763622283936
Epoch 2 Step 400: Loss = 0.11478538811206818 Accuracy = 0.9582294225692749
Epoch 2 Step 600: Loss = 0.03926467150449753 Accuracy = 0.960170567035675
Epoch 2 Step 800: Loss = 0.1360769271850586 Accuracy = 0.961649477481842
Epoch 2 Step 1000: Loss = 0.3363752067089081 Accuracy = 0.9621003866195679
Epoch 2 Step 1200: Loss = 0.08547146618366241 Accuracy = 0.9631036520004272
Epoch 2 Step 1400: Loss = 0.15003032982349396 Accuracy = 0.9639543294906616
Epoch 2 Step 1600: Loss = 0.15424351394176483 Accuracy = 0.9641044735908508
Epoch 2 Step 1800: Loss = 0.12477362900972366 Accuracy = 0.9645683169364929
End of epoch 2, loss: 0.03332900628447533, accuracy: 0.9652500152587891


### Exercise 4: Add Hidden Layers 

Next, you will add a couple of hidden layers to your model. Hidden layers help the model learn complex patterns in the data. 


In [17]:
from keras.layers import Input, Dense

# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation


In the above code: 

`Dense(64, activation='relu')` creates a dense (fully connected) layer with 64 units and ReLU activation function. 

Each hidden layer takes the output of the previous layer as its input.


### Exercise 5: Define the output layer 

Finally, you will define the output layer. Suppose you are working on a binary classification problem, so the output layer will have one unit with a sigmoid activation function. 


In [18]:
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

In the above code: 

`Dense(1, activation='sigmoid')` creates a dense layer with 1 unit and a sigmoid activation function, suitable for binary classification. 


### Exercise 6: Create the Model 

Now, you will create the model by specifying the input and output layers. 


In [19]:
model = Model(inputs=input_layer, outputs=output_layer)

In the above code: 

`Model(inputs=input_layer, outputs=output_layer)` creates a Keras model that connects the input layer to the output layer through the hidden layers. 


### Exercise 7: Compile the Model 

Before training the model, you need to compile it. You will specify the loss function, optimizer, and evaluation metrics. 


In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In the above code: 

`optimizer='adam'` specifies the Adam optimizer, a popular choice for training neural networks. 

`loss='binary_crossentropy'` specifies the loss function for binary classification problems. 

`metrics=['accuracy']` tells Keras to evaluate the model using accuracy during training. 


### Exercise 8: Train the Model 

You can now train the model on some training data. For this example, let's assume `X_train` is our training input data and `y_train` is the corresponding labels. 


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Input
import numpy as np

# Step 1: Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 2: Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10


I0000 00:00:1739879129.090384   16677 service.cc:146] XLA service 0x7f2728005c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739879129.090518   16677 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-02-18 15:15:29.143679: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-18 15:15:29.410092: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


13/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4737 - loss: 0.7006

I0000 00:00:1739879130.866777   16677 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.4757 - loss: 0.6991
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5669 - loss: 0.6853
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5642 - loss: 0.6883 
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5544 - loss: 0.6842 
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5722 - loss: 0.6828
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5325 - loss: 0.6850 
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5736 - loss: 0.6827
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5755 - loss: 0.6801 
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5604 - loss: 0.6817
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5753 - loss: 0.6767


In the above code: 

`X_train` and `y_train` are placeholders for your actual training data. 

`model.fit` trains the model for a specified number of epochs and batch size. 


### Exercise 9: Evaluate the Model 

After training, you can evaluate the model on test data to see how well it performs. 


In [22]:
# Example test data (in practice, use real dataset)
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - accuracy: 0.5554 - loss: 0.6931
Test loss: 0.6927566528320312
Test accuracy: 0.5600000023841858


In the above code: 

`model.evaluate` computes the loss and accuracy of the model on test data. 

`X_test` and `y_test` are placeholders for your actual test data. 


## Practice Exercises 

### Exercise 1: Basic Custom Training Loop 

#### Objective: Implement a basic custom training loop to train a simple neural network on the MNIST dataset. 

#### Instructions: 

- Set up the environment and load the dataset. 

- Define the model with a Flatten layer and two Dense layers. 

- Define the loss function and optimizer. 

- Implement a custom training loop to iterate over the dataset, compute the loss, and update the model's weights. 


In [23]:
# Write your code here
# Import necessary libraries
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function and Optimizer
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = keras.optimizers.Adam() 

# Step 4: Implement the Custom Training Loop
for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()}')

Epoch 1: Loss = 0.04005247354507446
Epoch 2: Loss = 0.022231586277484894
Epoch 3: Loss = 0.02454373612999916
Epoch 4: Loss = 0.014229792170226574
Epoch 5: Loss = 0.010417602024972439


<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function and Optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 

# Step 4: Implement the Custom Training Loop
for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()}')


### Exercise 2: Adding Accuracy Metric 

#### Objective: Enhance the custom training loop by adding an accuracy metric to monitor model performance. 

#### Instructions: 

1. Set up the environment and define the model, loss function, and optimizer. 

2. Add Sparse Categorical Accuracy as a metric. 

3. Implement the custom training loop with accuracy tracking.


In [25]:
# Write your code here
# Import necessary libraries
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), _ = keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = keras.optimizers.Adam() 
accuracy_metric = keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Training Loop with Accuracy Tracking
epochs = 5 
for epoch in range(epochs): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()} Accuracy = {accuracy_metric.result().numpy()}') 
    accuracy_metric.reset_state()

Epoch 1: Loss = 0.03396264836192131 Accuracy = 0.9238499999046326


2025-02-18 16:00:28.174748: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2: Loss = 0.044606201350688934 Accuracy = 0.9644500017166138
Epoch 3: Loss = 0.050628963857889175 Accuracy = 0.9762833118438721
Epoch 4: Loss = 0.0376456119120121 Accuracy = 0.983299970626831
Epoch 5: Loss = 0.019180921837687492 Accuracy = 0.9879666566848755


<details>
<summary>Click here for solution</summary><br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Training Loop with Accuracy Tracking
epochs = 5 
for epoch in range(epochs): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()} Accuracy = {accuracy_metric.result().numpy()}') 
    accuracy_metric.reset_state() 


### Exercise 3: Custom Callback for Advanced Logging 

#### Objective: Implement a custom callback to log additional metrics and information during training. 

#### Instructions: 

1. Set up the environment and define the model, loss function, optimizer, and metric. 

2. Create a custom callback to log additional metrics at the end of each epoch. 

3. Implement the custom training loop with the custom callback. 


In [26]:
# Write your code here
# Import necessary libraries
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense, Flatten 
from keras.callbacks import Callback 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    keras.Input(shape=(28, 28)),  # Updated Input layer syntax
    Flatten(), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = keras.optimizers.Adam() 
accuracy_metric = keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Callback
class CustomCallback(Callback): 
    def on_epoch_end(self, epoch, logs=None): 
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}') 

# Step 5: Implement the Custom Training Loop with Custom Callback
custom_callback = CustomCallback() 

for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    custom_callback.on_epoch_end(epoch, logs={'loss': loss.numpy(), 'accuracy': accuracy_metric.result().numpy()}) 
    accuracy_metric.reset_state()  # Updated method

End of epoch 1, loss: 0.03334770351648331, accuracy: 0.9211000204086304
End of epoch 2, loss: 0.016587262973189354, accuracy: 0.9641833305358887
End of epoch 3, loss: 0.013623653911054134, accuracy: 0.9762499928474426
End of epoch 4, loss: 0.010586993768811226, accuracy: 0.9826499819755554
End of epoch 5, loss: 0.013076844625175, accuracy: 0.9877833127975464


<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.callbacks import Callback 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    tf.keras.Input(shape=(28, 28)),  # Updated Input layer syntax
    Flatten(), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Callback
class CustomCallback(Callback): 
    def on_epoch_end(self, epoch, logs=None): 
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}') 

# Step 5: Implement the Custom Training Loop with Custom Callback
custom_callback = CustomCallback() 

for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    custom_callback.on_epoch_end(epoch, logs={'loss': loss.numpy(), 'accuracy': accuracy_metric.result().numpy()}) 
    accuracy_metric.reset_state()  # Updated method



### Exercise 4: Lab - Hyperparameter Tuning 

#### Enhancement: Add functionality to save the results of each hyperparameter tuning iteration as JSON files in a specified directory. 

#### Additional Instructions:

Modify the tuning loop to save each iteration's results as JSON files.

Specify the directory where these JSON files will be stored for easier retrieval and analysis of tuning results.


In [29]:
!pip install keras-tuner
# !pip install scikit-learn

In [31]:
# Write your code here
import json
import os
import keras_tuner as kt
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Step 1: Load your dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Step 2: Define the model-building function
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification example
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 3: Initialize a Keras Tuner RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials
    executions_per_trial=1,  # Set how many executions per trial
    directory='tuner_results',  # Directory for saving logs
    project_name='hyperparam_tuning'
)

# Step 4: Run the tuner search (make sure the data is correct)
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

# Step 5: Save the tuning results as JSON files
try:
    for i in range(10):
        # Fetch the best hyperparameters from the tuner
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        
        # Results dictionary to save hyperparameters and score
        results = {
            "trial": i + 1,
            "hyperparameters": best_hps.values,  # Hyperparameters tuned in this trial
            "score": None  # Add any score or metrics if available
        }

        # Save the results as JSON
        with open(os.path.join('tuning_results', f"trial_{i + 1}.json"), "w") as f:
            json.dump(results, f)

except IndexError:
    print("Tuning process has not completed or no results available.")

Reloading Tuner from tuner_results/hyperparam_tuning/tuner0.json


<details>
<summary>Click here for solution</summary> </br>

```python
!pip install keras-tuner
!pip install scikit-learn

import json
import os
import keras_tuner as kt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Step 1: Load your dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Step 2: Define the model-building function
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification example
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 3: Initialize a Keras Tuner RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials
    executions_per_trial=1,  # Set how many executions per trial
    directory='tuner_results',  # Directory for saving logs
    project_name='hyperparam_tuning'
)

# Step 4: Run the tuner search (make sure the data is correct)
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

# Step 5: Save the tuning results as JSON files
try:
    for i in range(10):
        # Fetch the best hyperparameters from the tuner
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        
        # Results dictionary to save hyperparameters and score
        results = {
            "trial": i + 1,
            "hyperparameters": best_hps.values,  # Hyperparameters tuned in this trial
            "score": None  # Add any score or metrics if available
        }

        # Save the results as JSON
        with open(os.path.join('tuning_results', f"trial_{i + 1}.json"), "w") as f:
            json.dump(results, f)

except IndexError:
    print("Tuning process has not completed or no results available.")
 ```   

</details>


### Exercise 5: Explanation of Hyperparameter Tuning

**Addition to Explanation:** Add a note explaining the purpose of num_trials in the hyperparameter tuning context:


<details>
<summary>Click here for solution</summary> </br>

```python
Explanation: "num_trials specifies the number of top hyperparameter sets to return. Setting num_trials=1 means that it will return only the best set of hyperparameters found during the tuning process."
 ```   

</details>


### Conclusion: 

Congratulations on completing this lab! You have now successfully created, trained, and evaluated a simple neural network model using the Keras Functional API. This foundational knowledge will allow you to build more complex models and explore advanced functionalities in Keras. 


Copyright © IBM Corporation. All rights reserved.
